In [1]:
!pip install -qUU  torch  transformers accelerate trl  peft datasets   --upgrade
#!pip install -qUU flash-attn --no-build-isolation
!pip uninstall torchvision torchaudio -y
#!pip install -qUU vllm 
#!pip install -qUU protobuf==4.25.3 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 4.7 MB/s eta 0:00:000:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 98.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 21.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 9.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import regex as re
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer
from trl import GRPOConfig, GRPOTrainer
from transformers import TrainerCallback

In [3]:
def get_dtype_config():
    """Return optimal dtype, training flag, and torch.compile support based on GPU capability."""
    if not torch.cuda.is_available():
        return torch.float32, {"bf16": False, "fp16": False}, False
    major, minor = torch.cuda.get_device_capability()
    if major >= 8:  # Ampere+ (A100, 3090, 4090, etc.)
        return torch.bfloat16, {"bf16": True, "fp16": False}, True
    elif major >= 7:  # Volta/Turing (V100, T4)
        return torch.float16, {"bf16": False, "fp16": True}, False
    else:
        return torch.float32, {"bf16": False, "fp16": False}, False

dtype, precision_args, use_compile = get_dtype_config()


dtype, precision_args, use_compile

(torch.float16, {'bf16': False, 'fp16': True}, False)

In [4]:
device_map="cuda:0"
dtype=torch.float32

## 1. Load model

In [5]:

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=dtype,
    device_map=device_map
    )

#if use_compile: 
#    model = torch.compile(model)

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/272 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [6]:
for p in model.parameters():
    break

p.device, p.dtype

(device(type='cuda', index=0), torch.float32)

## 2. SFT Warm-Start

The model gets near-zero reward with GRPO alone (~0.004 after 22 steps).
First fine-tune on a small subset of GSM8K gold chain-of-thought answers
so the model learns the reasoning format, then switch to GRPO.

In [7]:
def clean_gold_answer(answer_text):
    """Strip <<...>> annotations and reformat with 'The answer is: N.' ending."""
    parts = answer_text.split("####")
    reasoning = parts[0].strip()
    final_num = parts[1].strip() if len(parts) > 1 else ""
    reasoning = re.sub(r'<<.*?>>', '', reasoning)
    return f"{reasoning}\nThe answer is: {final_num}."

# Test it
test_ans = 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'
print(clean_gold_answer(test_ans))

Natalia sold 48/2 = 24 clips in May.
Natalia sold 48+24 = 72 clips altogether in April and May.
The answer is: 72.


In [8]:
sft_size = 1024
sft_subset = load_dataset("openai/gsm8k", "main")["train"].shuffle(seed=42).select(range(sft_size))

SYSTEM_PROMPT = (
    "You are a helpful math assistant. Solve the problem step by step, "
    "then give your final answer as a single number on the last line in "
    """this exact format:

        The answer is: {number}.
        """
)

def make_sft_example(ex):
    cleaned = clean_gold_answer(ex["answer"])
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": ex["question"]},
        {"role": "assistant", "content": cleaned},
    ]
    return {"messages": messages}

sft_ds = sft_subset.map(make_sft_example, remove_columns=sft_subset.column_names)
sft_ds

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 1024
})

In [9]:
precision_args

{'bf16': False, 'fp16': True}

In [10]:

sft_config = SFTConfig(
    output_dir="sft_gsm8k",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    logging_steps=10,
    save_steps=100,
    
    report_to='none',
    max_length=256,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,

    **precision_args,
)

sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=sft_ds,
    processing_class=tokenizer,

)

sft_trainer.train()

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Tokenizing train dataset:   0%|          | 0/1024 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1024 [00:00<?, ? examples/s]

Step,Training Loss
10,1.796393
20,1.270053
30,1.077552
40,1.053682
50,1.045025
60,1.020860
70,1.021932
80,1.014461


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=80, training_loss=1.162494707107544, metrics={'train_runtime': 339.0734, 'train_samples_per_second': 15.1, 'train_steps_per_second': 0.236, 'total_flos': 834647013310464.0, 'train_loss': 1.162494707107544})

Model is now SFT'd — it should produce reasoning + the expected format.
The same `model` object is updated in-place, so GRPO below starts from the SFT checkpoint.

In [11]:
model.eval()
model.config.use_cache = True

In [12]:
def format_prompt(question):
    system = (
        "You are a helpful math assistant. Solve the problem step by step, "
        "then give your final answer as a single number on the last line in "
        """this exact format:

        The answer is: {number}.
        """
    )
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": question},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


In [13]:
prompt = format_prompt("If a bag has 5 red and 3 blue marbles, how many marbles are there in total?")
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(out[0], skip_special_tokens=True))

system
You are a helpful math assistant. Solve the problem step by step, then give your final answer as a single number on the last line in this exact format:

        The answer is: {number}.
        
user
If a bag has 5 red and 3 blue marbles, how many marbles are there in total?
assistant
There are 5 + 3 = 8 marbles in the bag.
The answer is: 8.


Save the model and tokenizer.

In [14]:
model.save_pretrained("sft_gsm8k_checkpoint")
tokenizer.save_pretrained("sft_gsm8k_checkpoint")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('sft_gsm8k_checkpoint/tokenizer_config.json',
 'sft_gsm8k_checkpoint/chat_template.jinja',
 'sft_gsm8k_checkpoint/tokenizer.json')

Del the sft trainer and config.

In [15]:
del sft_config, sft_trainer
torch.cuda.empty_cache()

## 2. Dataset Setup

- Load GSM8K from HuggingFace (`datasets` library)
- Explore a few examples — understand the format (question + chain-of-thought + `#### answer`)
- Split into train/eval

In [16]:
model = AutoModelForCausalLM.from_pretrained(
    "sft_gsm8k_checkpoint",
    dtype=dtype,
    device_map=device_map
    )
tokenizer = AutoTokenizer.from_pretrained("sft_gsm8k_checkpoint")

Loading weights:   0%|          | 0/272 [00:00<?, ?it/s]

In [17]:
ds = load_dataset("openai/gsm8k", "main")
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

In [18]:
d  = ds['train'][0]
d

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [19]:
ans = d['answer'].split("####")[-1].strip()
ans

'72'

Func to fetch answers and save it in gold column of the dataset.

In [20]:
def extract_gold(ex):
    return {"gold": ex["answer"].split("####")[-1].strip()}

ds = ds.map(extract_gold, remove_columns=["answer"])
ds["train"][0]


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'gold': '72'}

Split test (1,319 examples) into two halves — 50% eval, 50% final test. Use train_test_split.

In [21]:
split = ds["test"].train_test_split(test_size=0.5, seed=42)
split

DatasetDict({
    train: Dataset({
        features: ['question', 'gold'],
        num_rows: 659
    })
    test: Dataset({
        features: ['question', 'gold'],
        num_rows: 660
    })
})

Update the dataset with spilt dataset.

In [22]:
ds["eval"] = split["train"]
ds["test"] = split["test"]
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'gold'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'gold'],
        num_rows: 660
    })
    eval: Dataset({
        features: ['question', 'gold'],
        num_rows: 659
    })
})

## 3. Reward Function

- Write `extract_answer(response)` — regex to pull the last number from model output
- Write `reward_fn(response, gold_answer)` — returns 1.0 if match, 0.0 if not
- Test it manually on the outputs already generated


**format_prompt** : Wraps a GSM8K question in a chat template with a system prompt: solve step-by-step, then output the final answer in a specific format.

**Formats tried:**
1. `#### <answer>` — model degenerated into repeating `####` endlessly
1. `Answer is {number}.` — model never produced this format → zero reward signal
1. `The answer is: {number}.` — model already tends to produce this from pretraining, giving immediate GRPO signal

**Takeaway:** Picked format the model can *already* partially produce, so GRPO has signal to work with from the start, selected the 3rd one.

In [23]:
d = ds['train'][0]
d

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'gold': '72'}

In [24]:
msg = [format_prompt(d['question']) for _ in range(32)]
gold = [d['gold'] for _ in range(32)]
print(msg[0])
len(msg)

<|im_start|>system
You are a helpful math assistant. Solve the problem step by step, then give your final answer as a single number on the last line in this exact format:

        The answer is: {number}.
        <|im_end|>
<|im_start|>user
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|im_end|>
<|im_start|>assistant



32

In [25]:
inp = tokenizer(msg, return_tensors="pt").to(model.device)
print(inp)

{'input_ids': tensor([[   1, 9690,  198,  ...,  520, 9531,  198],
        [   1, 9690,  198,  ...,  520, 9531,  198],
        [   1, 9690,  198,  ...,  520, 9531,  198],
        ...,
        [   1, 9690,  198,  ...,  520, 9531,  198],
        [   1, 9690,  198,  ...,  520, 9531,  198],
        [   1, 9690,  198,  ...,  520, 9531,  198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}


In [26]:
model.eval()

with torch.no_grad():
    out = model.generate(
    **inp,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=.9,
    pad_token_id=tokenizer.eos_token_id,
    use_cache=True
    )

out.shape

torch.Size([32, 176])

In [27]:
inp_shape = inp['input_ids'].shape[-1]
inp_shape

94

In [28]:
r = tokenizer.batch_decode(out, skip_special_tokens=True)
r

['system\nYou are a helpful math assistant. Solve the problem step by step, then give your final answer as a single number on the last line in this exact format:\n\n        The answer is: {number}.\n        \nuser\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\nassistant\nIn April, Natalia sold 48/2 = 24 clips\nIn May, Natalia sold 24/4 = 6 clips\nIn May, Natalia sold 6*48 = 284 clips\nIn total, Natalia sold 284+24=308 clips\nThe answer is: 308.',
 'system\nYou are a helpful math assistant. Solve the problem step by step, then give your final answer as a single number on the last line in this exact format:\n\n        The answer is: {number}.\n        \nuser\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\nassistant\nNatalia sold clips in April, and in May, she sold 48/2=

Extract the model's final numeric answer from its response and check if it used the expected format.

**Returns:** `(answer, has_format)` — a tuple of `int` (or `None`) and `bool`

**Logic:**
1. **Primary match** — regex looks for `The answer is: <number>` (case-insensitive). If found → return `(int, True)`
2. **Fallback** — grabs the last number in the entire string. If found → return `(int, False)`
3. **No match** — return `(None, False)`

**Edge cases handled:**
- Trailing dots: `72.` → `72` (via `int(float(...))`)
- Decimals: `72.0` → `72`
- Commas: `1,000` → `1000`
- Negative numbers: `-5`
- Multiple occurrences: takes the **last** match
- Case variations: `the answer is` / `The Answer Is` (via `re.IGNORECASE`)


In [29]:

def extract_answer(response):
    try:
        matches = re.findall(r'The answer is[:\s]*(\-?\d[\d,]*\.?\d*)', response, re.IGNORECASE)
        if matches:
            return int(float(matches[-1].replace(",", ""))), True
        nums = re.findall(r'\-?\d[\d,]*\.?\d*', response)
        if nums:
            return int(float(nums[-1].replace(",", ""))), False
    except Exception as e:
        print(f"[extract_answer error] response={response[:80]!r} err={e}")
    return None, False



In [30]:
assert extract_answer("The answer is: 72.")     == (72, True)
assert extract_answer("ans : 42")  == (42, False)
assert extract_answer("no numbers here")   == (None, False)

**Reward distribution**

| Answer | Format | Score |
|--------|--------|-------|
| ✅ Correct | ✅ `The answer is: {number}` | 1.0 |
| ✅ Correct | ❌ No format | 0.5 |
| ❌ Wrong | ✅ `The answer is: {number}` | 0.1 |
| ❌ Wrong | ❌ No format | 0.0 |
| `None` (no number found) | Either | 0.0 |

In [31]:
def reward_fn(completions, **kwargs):
    golds = kwargs["gold"]
    rewards = []
    for comp, gold in zip(completions, golds):
        try:
            ans, has_fmt = extract_answer(comp)
            gold_int = int(float(gold.replace(",", "")))
            if ans == gold_int and has_fmt:
                rewards.append(8.0)
            elif ans == gold_int and not has_fmt:
                rewards.append(3.2)
            elif ans != gold_int and has_fmt:
                closeness = max(0, 1 - abs(ans - gold_int) / max(abs(gold_int), 1))
                rewards.append(1.6 + 1.2 * closeness)
            elif ans is not None:
                closeness = max(0, 1 - abs(ans - gold_int) / max(abs(gold_int), 1))
                rewards.append(1.2 * closeness)
            else:
                rewards.append(0.0)
        except Exception as e:
            print(f"[reward_fn error] gold={gold!r} comp={comp[:80]!r} err={e}")
            rewards.append(0.0)
    return rewards


In [32]:
reward_fn(
    ["The answer is: 72.", "the answer is 72", "Answer is 99.", "wrong"],
    gold=["72", "72", "72", "72"]
)
# Expected: [1.0, 0.5, 0.1, 0.0]


[8.0, 8.0, 0.75, 0.0]

In [33]:
print(reward_fn(r, gold=gold))

[1.6, 1.85, 2.533333333333333, 2.1, 2.2, 1.6166666666666667, 1.9333333333333333, 2.1, 1.6, 2.1, 2.466666666666667, 2.1, 1.8, 1.8166666666666669, 2.4000000000000004, 2.4000000000000004, 2.2, 2.216666666666667, 2.2, 2.2333333333333334, 1.6, 1.6, 1.7000000000000002, 2.2, 1.6, 2.0666666666666664, 2.2, 2.1, 2.0, 2.2, 2.2, 2.6333333333333337]


## Dataset with formatting

In [34]:
ds = ds.rename_column("question", "prompt")
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'gold'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['prompt', 'gold'],
        num_rows: 660
    })
    eval: Dataset({
        features: ['prompt', 'gold'],
        num_rows: 659
    })
})

In [35]:
ds_rl = ds.map(lambda x: {"prompt": format_prompt(x["prompt"])})
ds_rl

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/660 [00:00<?, ? examples/s]

Map:   0%|          | 0/659 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'gold'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['prompt', 'gold'],
        num_rows: 660
    })
    eval: Dataset({
        features: ['prompt', 'gold'],
        num_rows: 659
    })
})

In [36]:
ds_rl['train'][0]

{'prompt': '<|im_start|>system\nYou are a helpful math assistant. Solve the problem step by step, then give your final answer as a single number on the last line in this exact format:\n\n        The answer is: {number}.\n        <|im_end|>\n<|im_start|>user\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|im_end|>\n<|im_start|>assistant\n',
 'gold': '72'}

## Perf cehck on eval dataset

In [37]:
from IPython.display import display, Markdown
from tqdm import tqdm

def perf_check(model, tokenizer):
    model.eval()
    model.config.use_cache = True
    tokenizer.padding_side = "left" # for training

    correct, total = 0, 0
    results = []
    batch_size = 64
    table_rows = []

    test_data = list(ds_rl["test"])
    for i in tqdm(range(0, len(test_data), batch_size), desc="Evaluating"):
        batch = test_data[i:i+batch_size]
        prompts = [ex["prompt"] for ex in batch]
        golds = [int(float(ex["gold"].replace(",", ""))) for ex in batch]
        
        inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(model.device)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=256)
        
        for j, (ids, gold_int) in enumerate(zip(out, golds)):
            response = tokenizer.decode(ids[inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
            ans, has_fmt = extract_answer(response)
            
            is_correct = (ans == gold_int)
            correct += int(is_correct)
            total += 1
            
            results.append({
                "gold": gold_int,
                "predicted": ans,
                "correct": is_correct,
                "response": response[:200],
            })
            table_rows.append(f"| {total} | {gold_int} | {ans} | {'✅' if is_correct else '❌'} | {response[:80].replace(chr(10), ' ')} |")

    #header = "| # | Gold | Pred | ✓ | Response |\n|---|------|------|---|----------|\n"
    #display(Markdown(header + "\n".join(table_rows)))
    print(f"\nFinal Accuracy: {correct}/{total} = {correct/total:.2%}")

    model.train()
    model.config.use_cache = False
    tokenizer.padding_side = "right" # for training 

    return table_rows 


In [38]:
_ = perf_check(model, tokenizer)

Evaluating: 100%|██████████| 11/11 [03:36<00:00, 19.64s/it]


Final Accuracy: 16/660 = 2.42%


## GRPO Loop


- For each prompt: generate G responses (start with G=4)
- Score each with your reward function
- Compute advantages (reward - group mean)
- Policy gradient update with clipping + KL penalty
- You already have a toy version in your `GRPO.ipynb` — adapt it to use the real model + real rewards


In [39]:
assert tokenizer.pad_token is not None

In [ ]:
import wandb

wandb.login()
wandb.init(project="grpo-gsm8k", name="grpo-smollm2-135m-sft-warmstart")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tripathysagar08 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [41]:
class VibecheckCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            reward = logs.get('reward')
            completions = logs.get('completions')
            if reward is not None:
                print(f"Step {state.global_step} | reward: {reward:.3f}")
            if completions and len(completions) > 0:
                print(f"Step {state.global_step} | Generated text (first example):\n{completions[0][:200]}...")


config = GRPOConfig(
    output_dir="grpo_gsm8k",
    num_generations=8,
    max_completion_length=256,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    #per_device_eval_batch_size=16,
    #eval_strategy="steps",
    #save_steps=50,
    #save_total_limit=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    logging_steps=10,
    report_to='wandb',
    beta=0.04,
    lr_scheduler_type="cosine",
    warmup_steps=20,
    gradient_checkpointing=False,
    weight_decay=0.01,
    eval_steps=50,

    **precision_args,

)


trainer = GRPOTrainer(
    model=model,
    reward_funcs=reward_fn,
    args=config,
    train_dataset=ds_rl["train"],
    #eval_dataset=ds_rl["eval"].select(range(32)),
    processing_class=tokenizer,
    callbacks=[VibecheckCallback()],
)

trainer.train()

Loading weights:   0%|          | 0/272 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'disable_compile'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


Step,Training Loss
10,0.182147
20,0.103409
30,0.116420
40,0.007909
50,0.064047
60,0.051535
70,0.056843
80,0.038035
90,0.079684
100,0.007830


Step 10 | reward: 1.570
Step 20 | reward: 1.637
Step 30 | reward: 1.871
Step 40 | reward: 1.770
Step 50 | reward: 1.884
Step 60 | reward: 1.912
Step 70 | reward: 1.905
Step 80 | reward: 1.943
Step 90 | reward: 1.911
Step 100 | reward: 1.971
Step 110 | reward: 2.004
Step 120 | reward: 1.919
Step 130 | reward: 1.942
Step 140 | reward: 1.971
Step 150 | reward: 2.038
Step 160 | reward: 1.787
Step 170 | reward: 1.844
Step 180 | reward: 1.918
Step 190 | reward: 2.121
Step 200 | reward: 1.966
Step 210 | reward: 2.056
Step 220 | reward: 1.963
Step 230 | reward: 2.071
Step 240 | reward: 1.893
Step 250 | reward: 2.005
Step 260 | reward: 2.086
Step 270 | reward: 1.922
Step 280 | reward: 2.016
Step 290 | reward: 1.964
Step 300 | reward: 1.963
Step 310 | reward: 1.989
Step 320 | reward: 2.067
Step 330 | reward: 1.955
Step 340 | reward: 1.895
Step 350 | reward: 1.961
Step 360 | reward: 2.103
Step 370 | reward: 1.983
Step 380 | reward: 1.917
Step 390 | reward: 1.986
Step 400 | reward: 2.016
Step 410 

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Step 1510 | reward: 1.966
Step 1520 | reward: 2.062
Step 1530 | reward: 2.021
Step 1540 | reward: 1.992
Step 1550 | reward: 2.101
Step 1560 | reward: 1.861
Step 1570 | reward: 1.990
Step 1580 | reward: 2.052
Step 1590 | reward: 2.198
Step 1600 | reward: 1.969
Step 1610 | reward: 2.027
Step 1620 | reward: 2.120
Step 1630 | reward: 1.904
Step 1640 | reward: 2.025
Step 1650 | reward: 2.069
Step 1660 | reward: 1.922
Step 1670 | reward: 1.975
Step 1680 | reward: 2.022
Step 1690 | reward: 2.119
Step 1700 | reward: 1.933
Step 1710 | reward: 1.998
Step 1720 | reward: 1.952
Step 1730 | reward: 1.997
Step 1740 | reward: 1.984
Step 1750 | reward: 2.121
Step 1760 | reward: 1.951
Step 1770 | reward: 1.969
Step 1780 | reward: 1.927
Step 1790 | reward: 2.012
Step 1800 | reward: 1.978
Step 1810 | reward: 1.995
Step 1820 | reward: 2.060
Step 1830 | reward: 2.022
Step 1840 | reward: 1.931
Step 1850 | reward: 1.991
Step 1860 | reward: 2.035


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Step 1868 | reward: 2.048


TrainOutput(global_step=1868, training_loss=0.030817893877170772, metrics={'train_runtime': 21171.087, 'train_samples_per_second': 0.353, 'train_steps_per_second': 0.088, 'total_flos': 0.0, 'train_loss': 0.030817893877170772})

## Stats after grpo

In [46]:
_ = perf_check(trainer.model, tokenizer)

Evaluating: 100%|██████████| 11/11 [03:36<00:00, 19.66s/it]


Final Accuracy: 17/660 = 2.58%


In [43]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (r

In [44]:
prompt = format_prompt("What is 3 + 5?")
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(out[0], skip_special_tokens=True))

system
You are a helpful math assistant. Solve the problem step by step, then give your final answer as a single number on the last line in this exact format:

        The answer is: {number}.
        
user
What is 3 + 5?
assistant
The answer is: 8.


In [61]:
def infer(model, tokenizer, prompt):
    #print(prompt)
    model.eval()
    model.config.use_cache = True
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs, 
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=.9,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=True
        )
    response = tokenizer.decode(out[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    model.train()
    model.config.use_cache = False
    return response


In [65]:
ds_rl['eval'][0]

{'prompt': '<|im_start|>system\nYou are a helpful math assistant. Solve the problem step by step, then give your final answer as a single number on the last line in this exact format:\n\n        The answer is: {number}.\n        <|im_end|>\n<|im_start|>user\nMadeline ate 6 grapes. Her brother wanted to make grape juice and used up 5 times as many grapes then Madeline to make a full glass. Their mother then used the remaining grapes to make 4 pies. How many grapes were there at the beginning if the pie recipe calls for 12 grapes per pie?<|im_end|>\n<|im_start|>assistant\n',
 'gold': '84'}

In [64]:
[infer(trainer.model, tokenizer, ds_rl['eval'][0]['prompt']) for i in range(5)]\
, ds_rl['eval'][0]['gold']

(['Madeline ate 6 x 12 = 72 grapes.\nHer brother wanted to make a full glass of juice and used up 5 times as many grapes as Madeline, so he used up 72 / 5 = 15 grapes.\nHe used up 15 / 6 = 3 grapes.\nThe pie recipe called for 4 pies.\nTherefore, the total number of grapes is 12 + 72 + 15 + 3 = 28 grapes.\nThe answer is: 28.',
  'Madeline ate 6 * 12 = 72 grapes.\nHer brother used up 5 * 12 = 60 grapes.\nSo, Madeline ate 72 - 60 = 12 grapes.\nShe made 4 * 12 = 48 grapes.\nShe used 12 grapes to make a full glass.\nTherefore, her mother used 48 + 12 = 60 grapes.\nThe answer is: 60.',
  "Madeline ate 6 x 12 = 72 grapes\nHer brother ate 5 x 12 = 60 grapes\nThe pie recipe calls for 12 grapes per pie\nMadeline's pie recipe calls for 12 grapes\nThe pie recipe calls for 72 grapes\nThe pie recipe calls for 4 pies\nThe pie recipe calls for 24 grapes\nThe answer is: 24.",
  'Madeline ate 6 * 12 = 72 grapes\nHer brother used up 5 * 12 = 60 grapes\nShe ate 72 - 60 = 14 grapes\nShe made 4 * 12 = 48 gr

## Push to hub

In [45]:
model.push_to_hub("tripathysagar/smollm2-135M-grpo-gsm8k")
tokenizer.push_to_hub("tripathysagar/smollm2-135M-grpo-gsm8k")

HfHubHTTPError: Client error '401 Unauthorized' for url 'https://huggingface.co/api/repos/create' (Request ID: Root=1-699b23cc-26e80b744687de7a27444f77;fb649835-457d-449d-94f3-1fe939039fec)
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401

Invalid username or password.